In [1]:
# Render offscreen -- make sure to set the PyOpenGL platform
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Dataset related imports 
from utils.data_loader_kine_adl import DATABASE_PATH, RightHand, LeftHand, load_subject_data
from utils.data_visualizer_kine_adl import RightHandJointNames, create_joint_state_msg

# Visualization improts 
from utils.pyrender_visualization import *


In [2]:
# subject_id = 12 #@param {type:"slider", min:0, max:22, step:1}
subject_id = 12 #@param {type:"slider", min:0, max:22, step:1}

# record_id = 106 #@param {type:"slider", min:101, max:129, step:1}
record_id = 129 #@param {type:"slider", min:101, max:129, step:1}


seq_to_predict = 200

In [3]:
print(DATABASE_PATH)
df = load_subject_data(database_path=DATABASE_PATH,
                        subject_id=subject_id, records_id=[record_id], load_anatomic_data=False)

kine-adl-be-uji_dataset/CSV DATA


In [4]:
df = df.rename(columns={"ID ":"ID"})

In [5]:
df

,EXPERIMENT,SUBJECT,R,ID,TIME,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,...,L_PIP3_F,L_MCP4_F,L_MCP34_A,L_PIP4_F,L_PalmArch,L_MCP5_F,L_MCP45_A,L_PIP5_F,L_WR_F,L_WR_A
0,1,12,129,86,0.970,-21.692934,15.005392,-3.448627,-1.898830,19.524729,...,20.427735,18.083420,3.757384,19.578716,11.861917,18.946576,-1.104062,24.377445,-3.120000,-10.478000
1,1,12,129,86,0.980,-21.693515,15.005668,-3.449044,-1.898827,19.524322,...,20.427703,18.083420,3.757384,19.578716,11.861917,18.951416,-1.105608,24.377445,-3.120000,-10.478000
2,1,12,129,86,0.990,-21.693970,15.005881,-3.449543,-1.898824,19.524777,...,20.427666,18.083420,3.757384,19.578716,11.861917,18.944201,-1.103303,24.377445,-3.120000,-10.478000
3,1,12,129,86,1.000,-21.694172,15.005969,-3.450107,-1.898822,19.526518,...,20.427628,18.083420,3.757384,19.578716,11.861917,18.919095,-1.095280,24.377445,-3.120000,-10.478000
4,1,12,129,86,1.010,-21.693955,15.005851,-3.450704,-1.898822,19.530046,...,20.427591,18.083420,3.757384,19.578716,11.861917,18.869237,-1.079348,24.377445,-3.120000,-10.478000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,1,12,129,87,24.191,-35.595795,14.674836,-8.619606,-8.228315,7.812995,...,13.813985,4.312934,6.549172,17.883662,8.914045,5.992500,-0.649753,20.321701,-1.549996,-11.279448
2323,1,12,129,87,24.201,-35.595762,14.674836,-8.619605,-8.228315,7.812984,...,13.783138,4.307442,6.549112,17.883883,8.958805,5.992500,-0.648306,20.320578,-1.561508,-11.279104
2324,1,12,129,87,24.211,-35.595735,14.674836,-8.619605,-8.228315,7.812993,...,13.773273,4.303701,6.548814,17.883479,8.989474,5.992500,-0.647121,20.319407,-1.569318,-11.278870
2325,1,12,129,87,24.221,-35.595713,14.674836,-8.619604,-8.228315,7.813012,...,13.776733,4.301339,6.548401,17.882741,9.008997,5.992500,-0.646199,20.318315,-1.574222,-11.278724


In [6]:
actions = list(range(101,133+1))+list(range(201,225+1))
len(actions)

58

In [7]:
import sys
print(sys.path)

sys.path.append('/home/plinardos/hand-motion-prediction/src')

['/home/plinardos/hand_synergies_study', '/home/plinardos/miniconda3/envs/visualization/lib/python37.zip', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/lib-dynload', '', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/site-packages', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/site-packages/IPython/extensions', '/home/plinardos/.ipython']


In [8]:
# Need to load all be_data to get the one_hot encodings
import torch
from data_utils import load_beuji_data
be_data = load_beuji_data( '/home/plinardos/hand-motion-prediction/data/beuji.pkl', 
                          range(1,22+1), 
                          actions, 
                          one_hot=True, 
                          total_frames = 100+200 )

Reading subject 1, R 101, ID 1
Reading subject 1, R 101, ID 2
This action is too brief with frame number 200. Consider combining phases or lowering the total frame requested if you want to use this sample.
Reading subject 1, R 102, ID 5
This action is too brief with frame number 261. Consider combining phases or lowering the total frame requested if you want to use this sample.
Reading subject 1, R 103, ID 8
Reading subject 1, R 103, ID 9
This action is too brief with frame number 100. Consider combining phases or lowering the total frame requested if you want to use this sample.
Reading subject 1, R 104, ID 11
Reading subject 1, R 104, ID 12
Reading subject 1, R 105, ID 13
This action is too brief with frame number 182. Consider combining phases or lowering the total frame requested if you want to use this sample.
Reading subject 1, R 106, ID 16
Reading subject 1, R 106, ID 17
Reading subject 1, R 106, ID 18
Reading subject 1, R 106, ID 19
Reading subject 1, R 107, ID 20
Reading subje

In [9]:
import importlib
import seq2seq_model
foo = importlib.reload(seq2seq_model)

class Object(object):
    pass
dataset = 'BEUJI'
FLAGS = Object()
if dataset == 'BEUJI':
    actions = list(range(101,133+1))+list(range(201,225+1))
    test_subject_ids = range(1,25)
    njoints = 36
    FLAGS.data_dir = "/home/plinardos/hand-motion-prediction/data/beuji.pkl"
if dataset == 'KINMUS':
    actions = range(1,26)
    test_subject_ids = range(1,22+1)
    njoints = 18
    FLAGS.data_dir = "/home/plinardos/hand-motion-prediction/data/kinmus.pkl"

one_hot = False
FLAGS.architecture = "tied"
FLAGS.seq_length_in = 100
FLAGS.seq_length_out = seq_to_predict
FLAGS.num_layers = 1
FLAGS.size = 1024
FLAGS.max_gradient_norm = 5
FLAGS.batch_size = 1
FLAGS.learning_rate = 0.005
FLAGS.learning_rate_decay_factor = 1
FLAGS.loss_to_use = "sampling_based"
FLAGS.omit_one_hot = False,
FLAGS.residual_velocities = False,
dtype = torch.float32

model = seq2seq_model.Seq2SeqModel(
    FLAGS.architecture,
    FLAGS.seq_length_in,
    FLAGS.seq_length_out,
    FLAGS.size, # hidden layer size
    FLAGS.num_layers,
    FLAGS.max_gradient_norm,
    FLAGS.batch_size,
    FLAGS.learning_rate,
    FLAGS.learning_rate_decay_factor,
    FLAGS.loss_to_use,
    number_of_actions = len( actions ),
    number_of_joints = njoints,
    one_hot = True,
    residual_velocities = FLAGS.residual_velocities,
    dtype=dtype)


One hot is  True
Input size is 94
rnn_size = 1024


In [10]:
pretrained_model_dir = '/home/plinardos/hand-motion-prediction/experiments/BEUJI/all/200/model_3000'
model.load_state_dict(torch.load(pretrained_model_dir))
model = model.cuda()
model.eval()

Seq2SeqModel(
  (cell): GRUCell(94, 1024)
  (fc1): Linear(in_features=1024, out_features=94, bias=True)
)

In [11]:
be_data[0][(1,106,16)].shape

(432, 94)

In [12]:
[i for i in zip(be_data[0].keys()) if i[0][1]==record_id]

[((1, 129, 86),),
 ((1, 129, 87),),
 ((2, 129, 86),),
 ((2, 129, 87),),
 ((3, 129, 86),),
 ((3, 129, 87),),
 ((4, 129, 86),),
 ((4, 129, 87),),
 ((5, 129, 86),),
 ((5, 129, 87),),
 ((6, 129, 86),),
 ((6, 129, 87),),
 ((7, 129, 86),),
 ((7, 129, 87),),
 ((8, 129, 86),),
 ((8, 129, 87),),
 ((9, 129, 86),),
 ((9, 129, 87),),
 ((10, 129, 86),),
 ((10, 129, 87),),
 ((11, 129, 86),),
 ((11, 129, 87),),
 ((12, 129, 86),),
 ((12, 129, 87),),
 ((13, 129, 86),),
 ((13, 129, 87),),
 ((14, 129, 86),),
 ((14, 129, 87),),
 ((15, 129, 86),),
 ((15, 129, 87),),
 ((16, 129, 86),),
 ((16, 129, 87),),
 ((17, 129, 86),),
 ((17, 129, 87),),
 ((18, 129, 86),),
 ((18, 129, 87),),
 ((19, 129, 86),),
 ((19, 129, 87),),
 ((20, 129, 86),),
 ((20, 129, 87),)]

In [13]:
be_data[0][(17, 101, 1)].shape

(1264, 94)

In [14]:
be_data[0][(1,106,16)][:,:36]

array([[ -0.94794588,   7.81322562,   4.59712388, ...,  12.18857918,
        -12.64500094,   1.92400003],
       [ -1.18478938,   8.11900684,   3.87262528, ...,  12.18852396,
        -12.64500216,   1.92400007],
       [ -1.39948605,   8.41681627,   3.16769878, ...,  12.18846272,
        -12.64500379,   1.92400013],
       ...,
       [-18.41393915,  22.95367592,  -4.86869809, ...,  55.78402046,
         -2.80853001,   5.37465762],
       [-18.83555807,  22.85470426,  -4.97929719, ...,  55.65608527,
         -2.70629827,   5.3713102 ],
       [-19.28013086,  22.7708929 ,  -5.06626067, ...,  55.56665981,
         -2.65469124,   5.36962038]])

In [16]:
import baselines

In [19]:
baselines.running_average()

TypeError: running_average() missing 3 required positional arguments: 'actions_dict', 'actions', and 'k'

In [ ]:
n_sequences = len( enc_in )
seq_length_out = dec_out[0].shape[0]

errs = dict()
# Make space for the error
errs[ action ] = np.zeros( (n_sequences, seq_length_out) )

# Get the lists for this action
enc_in, dec_in, dec_out = actions_dict[action]

# Iterate over batches, then take the mean of them
for i in np.arange( n_sequences ):

  # n, d = dec_out[i].shape

  # The last frame
  last_frame = dec_in[i][0, :]
  # last_frame[0:6] = 0

  if k > 1:
    # Get the last k-1 frames
    last_k = enc_in[i][(-k+1):, :]
    assert( last_k.shape[0] == (k-1) )

    # Merge and average them
    avg = np.mean( np.vstack( (last_k, last_frame) ), 0 )
  else:
    avg = last_frame

  # dec_out[i][:, 0:6] = 0
  idx_to_use = np.where( np.std( dec_out[i], 0 ) > 1e-4 )[0]

  ee = np.power( dec_out[i][:,idx_to_use] - avg[idx_to_use], 2 )
  # ee = np.sum( ee, 1 )
  # ee = np.sqrt( ee )
  # import pdb; pdb.set_trace()
  ee = np.mean(ee, 1)
  errs[ action ][i, :] = ee # this is an array of errors

errs[action] = np.mean( errs[action], 0 )

In [ ]:
set(df.ID)

In [ ]:
# dec_outs, preds = [], []
# phase = 17
# # for phase in set(df.ID):
# for i in range(0, df[df.ID==phase].shape[0],200):
#     print(i)

#     if i+300 > df[df.ID==phase].shape[0]:
#         final_frame = df[df.ID==phase].shape[0]
#         encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( be_data[0] , actions=[106], inference=((subject_id,record_id,phase), final_frame-300))
#         encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
#         decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
#         decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

#         step_preds = model(encoder_inputs, decoder_inputs)
#         print(decoder_outputs.squeeze().cpu().detach().numpy()[-(final_frame-(i+100)):,:].shape)
#         dec_outs.append(decoder_outputs.squeeze().cpu().detach().numpy()[-(final_frame-(i+100)):,:])  # keep only the new ones
#         preds.append(step_preds.squeeze().cpu().detach().numpy()[-(final_frame-(i+100)):,:])
#     else:
#         encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( be_data[0] , actions=[106], inference=((subject_id,record_id,phase),i))
#         encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
#         decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
#         decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

#         step_preds = model(encoder_inputs, decoder_inputs)
#         dec_outs.append(decoder_outputs.squeeze().cpu().detach().numpy())
#         preds.append(step_preds.squeeze().cpu().detach().numpy())
#         #     if i > 150:
#         #         break

# positions_test, positions_preds = np.row_stack(dec_outs), np.row_stack(preds)

In [ ]:
# subject_id = 12 #@param {type:"slider", min:0, max:22, step:1}
subject_id = 1 #@param {type:"slider", min:0, max:22, step:1}

# record_id = 106 #@param {type:"slider", min:101, max:129, step:1}
record_id = 101 #@param {type:"slider", min:101, max:129, step:1}

In [ ]:
dec_outs, preds = [], []
phase = 1
encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( be_data[0] , actions=[record_id], inference=((subject_id,record_id,phase),1))
encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

step_preds = model(encoder_inputs, decoder_inputs)
MSE = (step_preds-decoder_outputs).pow(2).mean()

# dec_outs = decoder_outputs.squeeze().cpu().detach().numpy()
preds = step_preds.squeeze().cpu().detach().numpy()
#     if i > 150:
#         break

positions_test, positions_preds = decoder_outputs.squeeze(), preds

In [ ]:
MSE

In [ ]:
decoder_inputs

In [ ]:
MSE

In [366]:
positions_preds.shape #(we are missing 400 that are the initial)

(200, 94)

In [367]:
positions_test.shape

(200, 94)

In [310]:
import pickle
with open('/home/plinardos/HandMo/BE_UJI/BE-UJI.pkl', "rb") as fp:
    loaded_data = pickle.load(fp)

In [314]:
positions_test[0,:36]

array([-41.07749188,  21.66048804,  15.51530496,  24.6850207 ,
        13.01273639,   5.92523169,  59.31905549,  49.03040579,
        33.58075884,  40.02430988,   3.3811079 ,  36.38236762,
        32.0947637 ,  31.97624649,   1.8268788 ,  20.04735255,
       -17.51665419,   6.78313083,  -3.24106108,  16.13736065,
        -6.5455792 ,  -4.75847481,  27.57931149,  -0.28934854,
        24.79996614,  28.14889028,  45.49439993,   9.46923043,
         6.58447568,  41.72923206,  16.20864526,   9.4160878 ,
        -0.10380534,  48.74504554,  -3.11998903,  -8.31700908])

In [312]:
positions_preds[0,:36]

array([-4.1197994e+01,  2.0988508e+01,  1.5293362e+01,  2.4388634e+01,
        1.2986641e+01,  6.3971767e+00,  5.9091957e+01,  4.8549839e+01,
        3.3190453e+01,  3.9646049e+01,  2.7502806e+00,  3.7558090e+01,
        3.0739908e+01,  3.2149872e+01,  1.6020612e+00,  2.1013613e+01,
       -1.7737375e+01,  6.9865890e+00, -2.7760029e+00,  1.6407616e+01,
       -6.3121138e+00, -4.5595808e+00,  2.7589243e+01, -3.5354084e-01,
        2.4980198e+01,  2.8558884e+01,  4.5127789e+01,  9.8392944e+00,
        6.0739384e+00,  4.1605148e+01,  1.6006533e+01,  9.6192017e+00,
       -4.5027826e-02,  4.8084019e+01, -2.9323964e+00, -8.4827452e+00],
      dtype=float32)

In [368]:
import pandas as pd
positions_test_df = pd.DataFrame(data = positions_test[:,:18], columns = jname_list)
positions_pred_df = pd.DataFrame(data = positions_preds[:,:18], columns = jname_list)

In [369]:
positions_test_df

,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,R_MCP23_A,R_PIP2_F,R_MCP3_F,R_PIP3_F,R_MCP4_F,R_MCP34_A,R_PIP4_F,R_PalmArch,R_MCP5_F,R_MCP45_A,R_PIP5_F,R_WR_F,R_WR_A
0,-34.266622,14.412244,-8.156372,-5.696734,10.156617,0.994792,18.812518,11.862211,7.637553,2.073787,5.143277,0.421330,29.839629,-0.640857,6.786193,0.677248,3.636056,9.193644
1,-34.269273,14.411283,-8.269785,-5.696781,10.156220,0.994939,18.975860,11.862190,7.633273,2.073775,5.141579,0.711653,29.838480,-0.640855,6.677900,0.758684,3.634563,9.208840
2,-34.272635,14.409743,-8.418435,-5.696821,10.155720,0.995115,19.188864,11.862159,7.641178,2.073754,5.144117,1.075167,29.845033,-0.640852,6.553091,0.880299,3.633918,9.223660
3,-34.276688,14.407494,-8.603407,-5.696848,10.155121,0.995317,19.452623,11.862115,7.667091,2.073725,5.152941,1.507276,29.862738,-0.640850,6.421118,1.049332,3.634677,9.236036
4,-34.281307,14.404421,-8.821841,-5.696850,10.154445,0.995536,19.762484,11.862057,7.717836,2.073684,5.170461,1.994393,29.895608,-0.640848,6.292541,1.271129,3.637528,9.243052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,26.180697,21.483044,-10.563912,-18.637261,53.263422,-2.543633,18.126307,70.401021,0.102210,43.466647,3.838710,4.778243,35.954642,25.719357,4.419177,10.799199,32.513916,1.377090
196,24.948407,21.176295,-9.416957,-17.265412,53.288080,-2.885442,18.618792,70.424493,0.439342,43.918615,3.787431,4.943330,34.975021,27.914806,3.866228,12.040168,32.510820,1.667562
197,23.450792,20.825195,-8.381315,-15.964256,53.307421,-3.227501,19.173634,70.455417,0.909973,44.490463,3.694895,5.232604,33.885022,30.246139,3.250269,13.509752,32.509468,1.947400
198,21.744654,20.439473,-7.478790,-14.754406,53.315377,-3.555161,19.811301,70.494190,1.529806,45.177602,3.563889,5.678538,32.737150,32.616574,2.593665,15.213734,32.510990,2.222215


In [370]:
positions_pred_df

,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,R_MCP23_A,R_PIP2_F,R_MCP3_F,R_PIP3_F,R_MCP4_F,R_MCP34_A,R_PIP4_F,R_PalmArch,R_MCP5_F,R_MCP45_A,R_PIP5_F,R_WR_F,R_WR_A
0,-33.645924,14.058371,-8.136568,-5.575007,9.952485,1.014913,18.094149,11.784414,7.277232,2.134656,4.442916,0.512753,28.725615,-0.416765,6.851607,1.225424,3.016482,9.196755
1,-33.042068,13.720672,-8.199422,-5.470879,9.746819,1.034550,17.498760,11.716785,6.913907,2.198992,3.740586,0.825342,27.628008,-0.184118,6.822983,1.815603,2.395228,9.219774
2,-32.459591,13.408088,-8.262539,-5.387537,9.545846,1.053731,16.912020,11.659074,6.561108,2.267664,3.044029,1.139769,26.563335,0.060476,6.786895,2.389999,1.774881,9.250975
3,-31.901756,13.124834,-8.326377,-5.324764,9.354194,1.071891,16.336697,11.610157,6.219363,2.340014,2.357068,1.457016,25.536737,0.319923,6.746513,2.942662,1.156983,9.291936
4,-31.369852,12.873186,-8.391624,-5.281424,9.175110,1.088385,15.774940,11.569099,5.889127,2.415108,1.683361,1.778036,24.550484,0.596569,6.704765,3.469161,0.543142,9.343711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-17.941290,28.909618,-13.436871,-0.423516,20.356922,-14.512301,-2.578673,21.495363,1.589306,24.124411,0.375334,19.964146,-7.452906,7.194288,13.142161,-1.180593,-12.425661,39.211544
196,-17.941877,28.885479,-13.815779,-0.373767,20.108147,-14.210187,-2.826465,21.269436,1.670317,24.160147,0.161982,20.193026,-7.565412,7.227477,13.138326,-1.381168,-12.378734,39.405411
197,-17.939013,28.863611,-14.209249,-0.321628,19.858992,-13.901511,-3.080223,21.036236,1.754384,24.200689,-0.054433,20.425011,-7.675111,7.266640,13.134345,-1.582948,-12.331584,39.595940
198,-17.932358,28.843876,-14.616770,-0.267159,19.609665,-13.586292,-3.339843,20.795948,1.840896,24.245800,-0.274270,20.660191,-7.782250,7.311454,13.129804,-1.785114,-12.284294,39.783581


In [247]:
# loaded_data[loaded_data.Subject==subject_id][loaded_data.R==record_id].iloc[:,5:23]

In [371]:
def load_data_get_positions(df_data):
    #  Load the data
    positions = None
    joint_names = None

    for iter in range(df_data.shape[0]):
        # Create joint state msg for RIGHT HAND
        js_right = create_joint_state_msg(df_data.iloc[iter, :], RightHand)

        positions = np.vstack([positions, js_right.position]) if positions is not None else js_right.position
        joint_names = js_right.name
    return(positions, joint_names)

In [372]:
pos_test, jname_test = load_data_get_positions(positions_test_df)
pos_pred, jname_pred = load_data_get_positions(positions_pred_df)

In [282]:
# pos_test, jname_test = load_data_get_positions(positions_test_df.iloc[:400,:])
# pos_pred, jname_pred = load_data_get_positions(positions_pred_df.iloc[:400,:])

In [333]:
pos_test.shape

(200, 22)

In [373]:
cfg = {joint_name: pos_test[:, i] for i, joint_name in enumerate(jname_test)}
pred_cfg = {joint_name: pos_pred[:, i] for i, joint_name in enumerate(jname_test)}

# pred_cfg = {joint_name: positions[:, i] * 1.3 for i, joint_name in enumerate(joint_names)}

loop_time = 0.01 * 4 * pos_test.shape[0] #positions_test_df.shape[0] # df.shape[0]

In [374]:
animation = fixed_prediction_animation(real_traj=cfg, pred_traj=pred_cfg, loop_time=loop_time,
                                real_color=np.array([58, 79, 87, 255]),
                                pred_color=np.array([117, 46, 18, 255]),
                                background_color=np.array([1.0, 1.0, 1.0]),
                                title=f"Subject {subject_id} - Record {record_id} \n Error {MSE}",
                                urdf_path='robots/right_hand_relative.urdf',
                                pred_hand_offset=0.2,
                                show=False)
clear_output()
animation

In [375]:
# Save animation to .mp4 file 
animation.save("./animations/Subject[%d]-Record[%d]-Predicting{%d}-Phase{%d}.mp4" % (subject_id, record_id, seq_to_predict, phase)) 

In [272]:
overlay_animation = fixed_prediction_animation(real_traj=cfg, pred_traj=pred_cfg, loop_time=loop_time,
                                real_color=np.array([58, 79, 87, 255]),
                                pred_color=np.array([117, 46, 18, 180]),
                                background_color=np.array([1.0, 1.0, 1.0]),
                                title="Subject %d - Record %d" % (subject_id, record_id),
                                urdf_path='robots/right_hand_relative.urdf',
                                pred_hand_offset=0.0,
                                show=False)
clear_output()
overlay_animation